# Stream Clustering

In [1]:
from scripts.preprocess_data import *
from scripts.preprocess_lemmatizer import preprocess_lemmatizer
from river import cluster
from river import stream
from river import feature_extraction
from river import compose
import nltk
#nltk.download('wordnet')


In [2]:
df = load_data()
df_sub = df.iloc[:1000]

In [3]:
preprocessed_text_sub = [ preprocess_lemmatizer(tweet)for tweet in df_sub['text']]


In [5]:
text_clust_algo = cluster.TextClust(real_time_fading=False, fading_factor=0.001, tgap=100, auto_r=False,
    radius=0.9)

stopwords = [ 'stop', 'the', 'to', 'and', 'a', 'in', 'it', 'is', 'I']

In [108]:
model = compose.Pipeline(
    feature_extraction.BagOfWords(lowercase=True, ngram_range=(1, 2), stop_words=stopwords),
    text_clust_algo,
)

In [ ]:
i=0
for text in preprocessed_text_sub:
    model.learn_one(text)
    i+=1
text_clust_algo.showclusters(topn=10,num=20)



In [16]:
text_clust_algo._agglomerative_clustering()

-------------------------------------------
Summary of micro clusters:
----
micro cluster id 0
micro cluster weight 490.25358925561653
weight: 88.32	 token: health
weight: 73.25	 token: politics
weight: 68.8	 token: business
-------------------------------------------


In [ ]:
text_clust_algo.learn_one(preprocessed_text_list_of_dicts[0])
text_clust_algo.micro_clusters

TypeError: can't multiply sequence by non-int of type 'float'